# Basic Motility Features Tutorial

Jake Lee, 2023-02-05, jake.h.lee@jpl.nasa.gov

This tutorial will cover how to calculate and save track motility features for basic machine learning. You are encourged to research, design, and implement additional features to make the task easier for machine learning.

Note that this step is **required** for traditional tabular classification, such as SVM, RandomForest, and neural networks. For methods that work directly on timeseries data, such as RNNs, this step is not required.

The output of this tutorial is already available as `train_basic_features.csv` and `test_basic_features.csv`.

## Importing Tracks

We will use the JSON format of the dataset, since it's easier to parse. The CSV may be more useful for gaining an intuitive understanding of the data and visualizing it.

In [1]:
import os
import os.path as op
from pathlib   import Path
from glob      import glob
from tqdm      import tqdm
from datetime  import datetime
from scipy     import stats

import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
import pandas as pd

In [2]:
###################################################
# !!! Remember to look at data_dictionary.txt !!! #
###################################################


# Set your target file here #######################

with open('../data/train.json', 'r') as f:
    track_data = json.load(f)

###################################################


# How many tracks are there?
print(f"n_tracks = {len(track_data.keys())}")

# What do the track Unique IDs (UIDs) look like?
track_uids = list(track_data.keys())
print(f"5 Example Track IDs = {track_uids[:5]}")

# What fields are avaiable for each track?
example_uid = track_uids[0]
print(f"Per-track keys = {track_data[example_uid].keys()}")

# What do the (t, x, y) track coordinates look like?
example_coords = track_data[track_uids[0]]['txy']
example_coords = np.array(example_coords)
np.set_printoptions(threshold=10)
print(f"Coordinate array = \n{example_coords}")

# What does the label look like?
example_label = track_data[track_uids[0]]['label']
print(f"Label = {example_label}")

n_tracks = 16080
5 Example Track IDs = ['lab_0_0', 'lab_0_1', 'lab_0_2', 'lab_0_3', 'lab_0_4']
Per-track keys = dict_keys(['txy', 'label'])
Coordinate array = 
[[  1.    184.166 463.817]
 [  2.    183.941 463.692]
 [  3.    183.716 463.567]
 ...
 [299.    146.777 448.634]
 [300.    146.795 448.518]
 [301.    146.813 448.402]]
Label = 0


## Implementing Features

Now that we can load the tracks, we need to calculate features. At its core, we need to calculate a single *number* that represents the *entire* track. The features we will implement here are:

* Average per-step speed
* Standard deviation of the per-step speed
* Physical length of the track
* Distance between the start and end of the track
* Time duration of the track

We will follow the same function pattern for each of these features to simplify the process.

In [3]:
def template_feature(coords):
    """Name of the Feature
    
    A short description of the feature goes here. Equations can be useful.
    
    Parameters
    ----------
    coords: array
        A numpy array containing the (t, x, y) coordinates of the track.
    
    Returns
    -------
    float
        The feature value for the entire array.
    
    """
    
    return 0

def mean_step_speed(coords):
    """Mean step speed of the entire track.
    
    The average per-step speed. Basically the average of distances between points adjacent in time.
    
    Returns
    -------
    float
        The average step speed.
    """

    speeds = []

    for i in range(1, coords.shape[0]):
        # Previous coordinate location
        prev = coords[i-1, 1:]
        # Current coordinate location
        curr = coords[i, 1:]
        
        # Speed in pixels per frame
        curr_speed = np.linalg.norm(curr - prev)
        
        # Accumulate per-step speeds into a list
        speeds.append(curr_speed)
    
    # Return the average of the speeds
    return np.mean(speeds)


def stddev_step_speed(coords):
    """Standard deviation of the step speed of the entire track.
    
    The standard deviation of the per-step speed.
    
    Returns
    -------
    float
        The stddev of the step speed.
    """

    speeds = []

    for i in range(len(coords)):
        # Previous coordinate location
        prev = coords[i-1]
        # Current coordinate location
        curr = coords[i]
        
        # Speed in pixels per frame
        curr_speed = np.linalg.norm(np.array(curr) - np.array(prev))
        
        # Accumulate per-step speeds into a list
        speeds.append(curr_speed)
    
    # Return the standard deviation of the speeds
    return np.std(speeds)


def track_length(coords):
    """Standard deviation of the step speed of the entire track.
    
    The standard deviation of the per-step speed.
    
    Returns
    -------
    float
        The length of the entire track.
    """

    lengths = []

    for i in range(1, coords.shape[0]):
        # Previous coordinate location
        prev = coords[i-1,1:]
        # Current coordinate location
        curr = coords[i,1:]
        
        # Speed in pixels per frame
        step_length = np.linalg.norm(curr - prev)
        
        # Accumulate per-step speeds into a list
        lengths.append(step_length)
    
    # Return the sum of the lengths
    return np.sum(lengths)


def e2e_distance(coords):
    """End-to-end distance of the track.
    
    The distance from the start and the end of the given track.
    
    Returns
    -------
    float
        The end-to-end distance of the entire track.
    """
    
    # Start and end of the track
    start = coords[0, 1:]
    end = coords[-1, 1:]
    
    # Return the distance
    return np.linalg.norm(end-start)


def duration(coords):
    """Duration of the track.
    
    The time duration of the track.
    
    Returns
    -------
    int
        The end-to-end duration of the entire track.
    """
    
    # Start and end times of the track
    start_t = coords[0, 0]
    end_t = coords[-1, 0]
    
    # Return the difference
    return end_t - start_t


######################################
# Implement your own features below! #
######################################

def chi(coords):
    start = coords[0]
    end = coords[len(coords)-1]
    i = 2
    use_slope = True
    while end[0]-start[0]==0:
        if(i>len(coords)):
            use_slope = False
            break
        end = coords[len(coords)-i]
        i+=1
    if use_slope:
        slope = float((end[1]-start[1])/(end[0]-start[0]))
    else:
        slope = 0
    err = 0
    for i in range(len(coords)):
        err += ((coords[i][2]-(slope*coords[i][1]+start[1]))**2)/len(coords)

    if math.isnan(err):
        return 0
    if math.isinf(err):
        return 1354300
        
    return err

def lin_reg(coords):
    coord_copy = np.array(coords)
    x = np.array(coord_copy[:,0]).reshape((-1, 1))
    y = np.array(coord_copy[:,1])
    model = LinearRegression().fit(x, y)
    return model.score(x, y)

def lin_reg_chi(coords):
    coord_copy = np.array(coords)
    x = np.array(coord_copy[:,0]).reshape((-1, 1))
    y = np.array(coord_copy[:,1])
    model = LinearRegression().fit(x, y)

    y_pred = model.predict(x)
    err = 0
    for i in range(len(coords)):
        err += ((y[i]-y_pred[i])**2)/len(coords)
        
    return err   

def squarederr(coords):
    start = coords[0, 1:]
    end = coords[-1, 1:]
    slope = float((end[1]-start[1])/(end[0]-start[0]))
    err = 0
    for i in range(0, coords.shape[0]):
        err += (coords[i][2]-(slope*coords[i][1]+start[1]))**2
    return err

def start(coords):
    start = coords[0, 1:]
    end = coords[-1, 1:]
    return start

def end(coords):
    start = coords[0, 1:]
    end = coords[-1, 1:]
    return end

## Implementing Feature cont.

Implementing more features will follow the same pattern as the functions above. You will either:

* Measure per-timestep metrics and calculate some statistic about the distribution
* Calculate some global feature over the entire track

There are many possible options for additional features. Consider visually plotting tracks to identify what geometric properties of a track indicate motility vs. non-motility.

## Generating the Feature CSV

Now that we have the feature implementations, we can extract features for every track and export it as a CSV. This is the CSV that will be used for training the machine learning model.

**NOTE:** You will also have to do this step when you're predicting the test set for submission to Kaggle.

In [4]:
######################################################
# !!! Set your list of implemented features here !!! #
######################################################

# FEATURE_LIST = [mean_step_speed, stddev_step_speed, track_length, e2e_distance, duration]
FEATURE_LIST = [chi]
TYPE = "train"
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILENAME = f"working/{TYPE}_features_{TIMESTAMP}.csv"
open(OUTPUT_FILENAME, "x")


######################################################
# You shouldn't have to modify the rest of this part #
######################################################

# Generate the feature csv
# header = ['uid', 'label']
# for featfunc in FEATURE_LIST:
#     header.append(featfunc.__name__)

# features = []

# track_uids = track_data.keys()
# for uid in track_uids:
#     curr_row = {
#         'uid': uid,
#         'label': track_data[uid]['label']
#     }
    
#     for featfunc in FEATURE_LIST:
#         curr_row[featfunc.__name__] = featfunc(np.array(track_data[uid]['txy']))
    
#     features.append(curr_row)

# with open(OUTPUT_FILENAME, 'w') as f:
#     writer = csv.DictWriter(f, fieldnames = header)
#     writer.writeheader()
#     for r in features:
#         writer.writerow(r)

# print("Written to:", OUTPUT_FILENAME)

data = []
track_uids = track_data.keys()
for uid in track_uids:
    dt = track_data[uid]['txy']
    data.append([str(uid), track_data[uid]['label'], chi(dt), lin_reg(dt), lin_reg_chi(dt), stddev_step_speed(dt)])

data = pd.DataFrame(data)


data[2] = data[2] /data[2].abs().max()
data[3] = data[3] /data[3].abs().max()
data[4] = data[4] /data[4].abs().max()
data[5] = data[5] /data[5].abs().max()


data.rename(columns={0: 'uid', 1: 'label', 2: 'chi', 3: 'lin_reg', 4: 'lin_reg_chi', 5: 'stddev_step_speed'}, inplace=True)

print(data)

data.to_csv('out.csv', index=False)


c:\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Python39\lib\site-packages\sklea

              uid  label       chi   lin_reg  lin_reg_chi  stddev_step_speed
0         lab_0_0      0  0.000920  0.992533     0.000065           0.154567
1         lab_0_1      0  0.000228  0.973844     0.000128           0.154105
2         lab_0_2      0  0.000256  0.894242     0.000435           0.154033
3         lab_0_3      0  0.000088  0.967026     0.000379           0.154711
4         lab_0_4      0  0.006075  0.000296     0.000100           0.153187
...           ...    ...       ...       ...          ...                ...
16075  sim_419_59      0  0.005071  0.999245     0.000057           0.160588
16076  sim_419_60      0  0.014942  0.999707     0.000089           0.232843
16077  sim_419_61      0  0.024277  0.998892     0.000236           0.211919
16078  sim_419_62      0  0.007277  0.998953     0.000045           0.145301
16079  sim_419_63      0  0.006620  0.999056     0.000480           0.267385

[16080 rows x 6 columns]
